In [ ]:
import os
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

### MovieLens 데이터 불러오기

In [ ]:
# Google colab 사용할 때, 변경하기
path = '../data/ml-latest-small/'
os.listdir(path)

In [ ]:
tags_df = pd.read_csv(os.path.join(path, 'tags.csv'), encoding='utf-8')
ratings_df = pd.read_csv(os.path.join(path, 'ratings.csv'), index_col = 'userId', encoding='utf-8')
movies_df = pd.read_csv(os.path.join(path, 'movies.csv'), index_col = 'movieId', encoding='utf-8')

In [ ]:
print(ratings_df.shape)
print(movies_df.shape)

### 장르 분석하기

* 전체 장르 파악하기

In [ ]:
movies_df.head()

In [ ]:
all_genres = [x.split('|') for x in movies_df['genres'].values]

In [ ]:
import itertools
genres = list(set(list(itertools.chain(*all_genres))))
print(len(all_genres))
print(len(genres))
print(genres)

In [ ]:
genres_df = pd.DataFrame(columns=genres, index=movies_df.index)
genres_df.head()

In [ ]:
ct = 0
for i, row in genres_df.iterrows():
    movie_id = row.name
    list_of_genres = movies_df.loc[movie_id]['genres'].split('|')
    
    genres_df.loc[movie_id][list_of_genres] = 1

In [ ]:
genres_df = genres_df.fillna(0)
genres_df['num_genres'] = genres_df.sum(axis=1)
print(genres_df.shape)
print(genres_df)

* get_dummies 활용하기

In [ ]:
genres_df = movies_df['genres'].str.get_dummies(sep='|')
genres_df.head()

In [ ]:
movies_df = pd.concat([movies_df, genres_df], axis=1)
movies_df.head()

### 특정 장르의 평점과 user 분석

In [ ]:
movies_df.columns

In [ ]:
movieId_list = movies_df['title'][movies_df.Animation == 1]
movieId_list.index


In [ ]:
animation_df = ratings_df[ratings_df['movieId'].isin(movieId_list.index)]
animation_df

In [ ]:
# user는 animation장르의 영화에 대해 다음과 같은 평점을 부여
animation_df.groupby('userId')['rating'].mean()

### 장르간 상관관계

In [ ]:
genres_df.corr()

In [ ]:
plt.figure(figsize=(40,40))
sns.heatmap(genres_df.corr(), annot=True)

### 영화 이름, 연도 분석

In [ ]:
movies_df.head()

In [ ]:
title_df = movies_df[['title', 'genres']]
title_df.head()

In [ ]:
title_df['year'] = title_df['title'].str.extract('(\(\d\d\d\d\))')
title_df.head()

* 혹시 연도가 제대로 표시가 안되어있는 영화도 있을 수 있으니 nan 확인

In [ ]:
title_df.isna().sum()

In [ ]:
title_df[title_df['year'].isnull()]

In [ ]:
# 연도가 없는 영화 또는 (no genres listed)라고 표시된 영화는 드랍할 수 있다.
title_df[title_df.genres == '(no genres listed)'].shape

In [ ]:
title_df.dropna(axis=0, inplace=True)
title_df['year'] = title_df['year'].apply(lambda x: x.replace('(','').replace(')',''))
title_df.head()

### 연도별 영화 데이터 분석

In [ ]:
year_freq_df = title_df.groupby('year')['title'].count()
year_freq_df

In [ ]:
year_freq_df.sort_values(ascending=False)

In [ ]:
year_freq_df.describe()

### 영화와 개봉연도, 그리고 평점

In [ ]:
movies_df.head()

In [ ]:
title_df

In [ ]:
title_df[title_df['year'] == '2017']

In [ ]:
# 2017년 영화 평점 평균
ratings_df['rating'][ratings_df['movieId'].isin(title_df[title_df['year'] == '2017'].index)].mean()

In [ ]:
results = []
for year in title_df['year'].unique():
    avg_ratings = ratings_df['rating'][ratings_df['movieId'].isin(title_df[title_df['year'] == str(year)].index)].mean()
    results.append((year, avg_ratings))

result_df = pd.DataFrame(results, columns=['year','avg_ratings'])
result_df.sort_values(by='year')

In [ ]:
result_df.hist()